In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as color

Solving environment: done

# All requested packages already installed.



In [2]:

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [3]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [5]:
df.shape

(180, 3)

In [6]:
df_geo_coor = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.drop(0,inplace=True)
df_toronto=df_toronto.dropna()
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [10]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [11]:
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc

In [13]:
CLIENT_ID = 'D5KEKFMMQMXTMOEFE2QEO4O0VJACVBKTF0B4NGI1KRX121BO'
CLIENT_SECRET ='4MJI4KE33ZCYAU3W2DPOS2044POVITE1HJN5PBIDU2S3RGYT'
VERSION = '20180604'

In [14]:
neighborhood_name = df_toronto_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")
neighborhood_latitude = df_toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

The first neighborhood's name is 'The Beaches'.
Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                    'Venue Category']
    
    return(nearby_venues)

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_denc['Borough'].unique()),
        df_toronto_denc.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [19]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
df_toronto_denc.groupby('Neighborhood').count()

,PostalCode,Borough,Latitude,Longitude
Neighborhood,,,,
Berczy Park,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",1,1,1,1
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1
Central Bay Street,1,1,1,1
Christie,1,1,1,1
Church and Wellesley,1,1,1,1
"Commerce Court, Victoria Hotel",1,1,1,1
Davisville,1,1,1,1


In [21]:
toronto_denc_onehot = pd.get_dummies(df_toronto_denc[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = df_toronto_denc['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Neighborhood,Berczy Park,"Brockton, Parkdale Village, Exhibition Place","Business reply mail Processing Centre, South Central Letter Processing Plant Toronto","CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Central Bay Street,Christie,Church and Wellesley,"Commerce Court, Victoria Hotel",Davisville,...,St. James Town,"St. James Town, Cabbagetown",Stn A PO Boxes,Studio District,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park","The Annex, North Midtown, Yorkville",The Beaches,"The Danforth West, Riverdale","Toronto Dominion Centre, Design Exchange","University of Toronto, Harbord"
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"India Bazaar, The Beaches West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Studio District,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_denc_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
toronto_denc_grouped.head()

,Neighborhood,Berczy Park,"Brockton, Parkdale Village, Exhibition Place","Business reply mail Processing Centre, South Central Letter Processing Plant Toronto","CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Central Bay Street,Christie,Church and Wellesley,"Commerce Court, Victoria Hotel",Davisville,...,St. James Town,"St. James Town, Cabbagetown",Stn A PO Boxes,Studio District,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park","The Annex, North Midtown, Yorkville",The Beaches,"The Danforth West, Riverdale","Toronto Dominion Centre, Design Exchange","University of Toronto, Harbord"
0,Berczy Park,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Brockton, Parkdale Village, Exhibition Place",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Business reply mail Processing Centre, South C...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Bay Street,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Berczy Park,Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village",Davisville
1,"Brockton, Parkdale Village, Exhibition Place","Brockton, Parkdale Village, Exhibition Place","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
2,"Business reply mail Processing Centre, South C...","Business reply mail Processing Centre, South C...","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
3,"CN Tower, King and Spadina, Railway Lands, Har...","CN Tower, King and Spadina, Railway Lands, Har...","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
4,Central Bay Street,Central Bay Street,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [24]:
kclusters = 5

toronto_denc_grouped_clustering = toronto_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [25]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = df_toronto_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,The Beaches,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,"The Danforth West, Riverdale","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,"India Bazaar, The Beaches West","University of Toronto, Harbord",Davisville North,"High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village",Davisville
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Studio District,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Lawrence Park,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [color.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,The Beaches,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
1,East Toronto,0,"The Danforth West, Riverdale","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
2,East Toronto,0,"India Bazaar, The Beaches West","University of Toronto, Harbord",Davisville North,"High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village",Davisville
3,East Toronto,0,Studio District,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
4,Central Toronto,0,Lawrence Park,"University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
5,Central Toronto,0,Davisville North,"University of Toronto, Harbord","India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village",Davisville
6,Central Toronto,0,"North Toronto West, Lawrence Park","University of Toronto, Harbord",Lawrence Park,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
7,Central Toronto,0,Davisville,Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village","University of Toronto, Harbord"
8,Central Toronto,0,"Moore Park, Summerhill East","University of Toronto, Harbord",Lawrence Park,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"
9,Central Toronto,0,"Summerhill West, Rathnelly, South Hill, Forest...","University of Toronto, Harbord",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [32]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Downtown Toronto,1,"Queen's Park, Ontario Provincial Government","University of Toronto, Harbord",Lawrence Park,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [33]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Downtown Toronto,2,"Richmond, Adelaide, King","University of Toronto, Harbord",Lawrence Park,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [34]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,3,"Toronto Dominion Centre, Design Exchange",Lawrence Park,"Kensington Market, Chinatown, Grange Park","India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village"


In [35]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,West Toronto,4,"Little Portugal, Trinity",Davisville North,"India Bazaar, The Beaches West","High Park, The Junction South","Harbourfront East, Union Station, Toronto Islands","Garden District, Ryerson","Forest Hill North & West, Forest Hill Road Park","First Canadian Place, Underground city","Dufferin, Dovercourt Village",Davisville
